In [3]:
import numpy as np
import pandas as pd
from numba import njit, prange
from interpolation import interp

In [4]:
γ = 0.5772156649

In [6]:
f = open("/Users/hyde/Dropbox/UArizona/ECON 696/data_for_bellman_computations.csv")
lines = f.readlines()

In [7]:
n_naics_recode = 7
n_region = 10
n_lag_inv = 2
n_lag2_inv = 2
n_ordered_violator = 3
n_DAV = 20

simulated_dist = np.empty((n_naics_recode, n_region, n_lag_inv, n_lag2_inv, n_ordered_violator, n_DAV, 80, 7))

for line in lines[1:]:
    line = line.split(',')
    for i in range(80):
        simulated_dist[int(line[0])-1, int(line[1])-1, int(line[2]), int(line[3]), int(line[4]), int(line[5])-1][i] = [float(e) for e in line[7+7*i:14+7*i]]

In [25]:
def BGL(θ):

    θ_I, θ_F, θ_V, θ_H, θ_X = θ
    
    # depreciation rate
    δ = 0.1
    
    # discount factor
    β = 0.95 ** (1/4.)
    
    # focus on the fist sector
    sector = 0
    
    # initialize V(Ω') and V(Ω)
    # set the initial values to be 0

    # region (10) * lag_investment (2) * lag2_investent(2) * ordered_violator (3) * disc_violations (20)
    n_region = 10
    n_Linv = 2
    n_L2inv = 2
    n_ordered_violator = 3
    n_DAV = 20
    n_vio = 2
    
    DAV_range = np.arange(0., 10.0, 0.5)
    
    def initialize():
        # V(Ω')
        VΩ_prime = np.zeros((n_region, n_Linv, n_L2inv, n_ordered_violator, n_DAV, n_vio))
        # V(Ω) 
        VΩ = np.zeros((n_region, n_Linv, n_L2inv, n_ordered_violator, n_DAV))
        
        return VΩ_prime, VΩ
    
    @njit(parallel=True)
    def VΩ_prime_operator(VΩ_prime, VΩ):
        
        for region in prange(n_region):
            for Linv in prange(n_Linv):
                for L2inv in prange(n_L2inv):
                    for ordered_violator in prange(n_ordered_violator):
                        for i_DAV in prange(n_DAV):
                            for vio in prange(n_vio):
                                DAV = (i_DAV - 1) * 0.5

                                # for compliance
                                if ordered_violator == 0:
                                    VΩ_prime[region, Linv, L2inv, ordered_violator, i_DAV, vio] = β * VΩ[region, 0, Linv, ordered_violator, i_DAV] + γ

                                else:

                                    DAV_next = (1 - δ) * DAV + vio

                                    VΩ_prime[region, Linv, L2inv, ordered_violator, i_DAV, vio] = np.log(np.exp(β * interp(DAV_range, VΩ[region, 0, Linv, ordered_violator, :], DAV_next))
                                                                                                   + np.exp(θ_X + β * interp(DAV_range, VΩ[region, 1, Linv, ordered_violator, :], DAV_next))) + γ



#     @njit(parallel=True)
    def VΩ_operator(VΩ_prime, VΩ):
        
        for region in prange(n_region):
            for Linv in prange(n_Linv):
                for L2inv in prange(n_L2inv):
                    for ordered_violator in prange(n_ordered_violator):
                        for i_DAV in prange(n_DAV):
                            DAV = (i_DAV - 1) * 0.5
                            
                            # find out the transitions
                            # for compliance
                            if (ordered_violator == 0):
                                transition = simulated_dist[sector, region, 0, 0, 0, 0]
                            else:
                                transition = simulated_dist[sector, region, Linv, L2inv, ordered_violator, i_DAV]
                            
                            VΩ[region, Linv, L2inv, ordered_violator, i_DAV] = 0.
                            
                            for i in prange(80):
                                ins, vio, fine, prob, prob0, prob1, prob2 = transition[i]
                                util = θ_I * ins + fine * θ_F + vio * θ_V + (ordered_violator == 2) * θ_H
                                
                                # if transit to compliance plant
                                util += prob0 * VΩ_prime[region, Linv, L2inv, 0, 0, int(vio)]
                                
                                # if transit to regular violator or high priority violator
                                
                                util += prob1 * interp(DAV_range, VΩ_prime[region, Linv, L2inv, 1, :, int(vio)], DAV)
                                util += prob2 * interp(DAV_range, VΩ_prime[region, Linv, L2inv, 2, :, int(vio)], DAV)
                                
                                VΩ[region, Linv, L2inv, ordered_violator, i_DAV] += prob * util
                        

    return initialize, VΩ_prime_operator, VΩ_operator

In [26]:
initialize, VΩ_prime_operator, VΩ_operator = BGL((-2., -0.5, -0.5, -0.1, -0.1))

In [27]:
VΩ_prime, VΩ = initialize()

In [28]:
VΩ_operator(VΩ_prime, VΩ)

In [14]:
VΩ_prime_operator(VΩ_prime, VΩ)

In [ ]:
for i in range(100):
    
    VΩ_prime_old = np.copy(VΩ_prime)

    VΩ_operator(VΩ_prime, VΩ)
    VΩ_prime_operator(VΩ_prime, VΩ)
    
    dist = np.linalg.norm(VΩ_prime_old - VΩ_prime)
    print(dist)

174.54606039143127
140.59478908265984
105.18256552376845
75.7245540293978
54.163597123298416
41.46525873394979
36.70714002035848
36.51729495967333
37.76965476918309
39.15718035000848
40.47029674123096
41.82673547827424
43.36200266034836
45.15742896002392
47.24296845074155
49.61537325073562
52.25490744283916
55.13565106536432
58.22956658796471
61.507120373174715
64.93720927237959
68.48756832540737
72.12521815379037
75.81614443342468
79.52537817398989
83.21748651980103
86.85611541697682
90.40478234869293
93.83069357112444
97.10765141729084
100.21750731662344
103.14857432602336
105.89433667159756
108.45108280874231
110.81633764061483
112.98845463087666
114.9674329576517
116.75570879662767


In [137]:
dist = 0.
while dist > 1e-7:
    
    VΩ_prime_old = np.copy(VΩ_prime)

    VΩ_prime_operator(VΩ_prime, VΩ)
    VΩ_operator(VΩ_prime, VΩ)
    
    dist = np.linalg.norm(VΩ_prime_old - VΩ_prime)
    print(dist)

In [113]:
VΩ_prime

array([[[[[ 41.84297502,  41.82135946,  41.79951029, ...,
            41.4518976 ,  41.43720349,  41.42330871],
          [379.01754814, 364.66464183, 350.31173553, ...,
           157.51463347, 150.90100929, 144.77734044],
          [115.111595  , 127.14563733, 139.17967967, ...,
           156.23737364, 150.82237444, 145.27235948]],

         [[ 41.84297502,  41.82135946,  41.79951029, ...,
            41.4518976 ,  41.43720349,  41.42330871],
          [379.01754814, 364.66464183, 350.31173553, ...,
           157.51463347, 150.90100929, 144.77734044],
          [115.111595  , 127.14563733, 139.17967967, ...,
           156.23737364, 150.82237444, 145.27235948]]],


        [[[ 40.93915765,  40.91361993,  40.88814489, ...,
            40.58082073,  40.56897105,  40.5576588 ],
          [352.0455364 , 335.68422293, 319.32290946, ...,
           152.65116953, 147.74962789, 142.69109526],
          [130.63980445, 141.9478855 , 153.25596654, ...,
           155.80578436, 150.24174561, 1